# Train model notebook

Source: 

- https://joeddav.github.io/blog/2020/05/29/ZSL.html
- https://huggingface.co/docs/setfit/how_to/zero_shot
- : Generic model 

Dataset:

- https://huggingface.co/docs/datasets/create_dataset

In [6]:
import pandas as pd
import numpy as np
import re


ACRONYM = ["YTA", "YWBTA", "NTA", "YWNBTA", "ESH", "NAH", "INFO"]

## Data enginerring
data = pd.read_parquet("data/raw/comments-25-06-2024-14h30.parquet")
data = data[~data.body.isna()]

data["label"] = True # TODO: fake label
data["label"] = data["label"].map(lambda x: np.random.randint(0, 2) == 1)

def processing(doc):
    doc = doc.lower()
    doc = re.sub(r'http\S+|www\S+|https\S+', '', doc, flags=re.MULTILINE)
    # Suppression des mentions d'utilisateurs (par exemple, u/username)
    doc = re.sub(r'u/\S+', 'this people', doc)
    # Suppression des caractères spéciaux et de la ponctuation
    doc = re.sub(r'\W', ' ', doc)
    doc = re.sub(r'  ', ' ', doc)
    return doc

In [ ]:
# source: https://huggingface.co/blog/setfit

from sentence_transformers.losses import CosineSimilarityLoss
from setfit import SetFitModel, SetFitTrainer
from datasets import Dataset


MODEL ="sentence-transformers/paraphrase-albert-small-v2" 
# "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"

train_ds = Dataset.from_dict({"text": data["body"].map(processing), "label": data["label"]}, split="test")
test_ds = Dataset.from_dict({"text": data["body"].map(processing), "label": data["label"]}, split="train")

model = SetFitModel.from_pretrained(MODEL)

trainer = SetFitTrainer(
    model=model,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    loss_class=CosineSimilarityLoss,
    batch_size=16,
    num_iterations=20, # Number of text pairs to generate for contrastive learning
    num_epochs=1 # Number of epochs to use for contrastive learning
)

In [8]:

# source: https://huggingface.co/docs/setfit/how_to/zero_shot

# train_dataset = Dataset.from_dict({"text": data["body"], "label": data["label"]}, split="test").with_format("torch")
# dataloader = DataLoader(train_dataset, batch_size=4)
# train_dataloader = DataLoader(dataloader, shuffle=True, batch_size=16)


/home/romain/miniconda3/envs/projet/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [10]:
trainer.train()

***** Running training *****
  Num unique pairs = 9320
  Batch size = 16
  Num epochs = 1
  Total optimization steps = 583
  0%|          | 0/583 [00:00<?, ?it/s]


                                             

                                         



{'embedding_loss': 0.2187, 'learning_rate': 3.3898305084745766e-07, 'epoch': 0.0}


{'embedding_loss': 0.2741, 'learning_rate': 1.694915254237288e-05, 'epoch': 0.09}


{'embedding_loss': 0.2755, 'learning_rate': 1.8435114503816797e-05, 'epoch': 0.17}
